In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from typing import *
from IPython.display import Image, display, clear_output
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import math 
from torch import nn, Tensor
from torch.nn.functional import softplus, relu
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from functools import reduce

from plotting import make_vae_plots

CMNIST_list = torch.load("data/ColoredMNIST/train1.pt")
CMNIST = pd.DataFrame(CMNIST_list, columns = ["image", "label"])

images = CMNIST.iloc[:,0]
labels_df = CMNIST.iloc[:,1]
labels = torch.tensor(labels_df)

In [ ]:
from torchvision.transforms import ToTensor

transform_to_tensor = ToTensor()
transform_to_tensor(images[0]).shape

# Plotting function

In [ ]:
def show_CMNIST(image, label, environment):
    """Show image with landmarks"""
    plt.figure
    plt.imshow(image)
    plt.text(1,2,"Label: {}".format(label), backgroundcolor = "white",
             color = "black", fontsize = 6)
    plt.text(1,5,"Environment: {}".format(environment), backgroundcolor = "white",
             color = "black", fontsize = 6)
    plt.axis('off')
    plt.show
    plt.pause(0.001)  # pause a bit so that plots are updated

show_CMNIST(images[0], labels[0], None)

In [ ]:
class CMNISTDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, pt_file, environment=None, transform=None):
        """
        Args:
            pt_file (string): Path to the pt file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            environment (integer): Integer indicating the environment the data comes from
        """
        self.CMNIST_frame = pd.DataFrame(torch.load(pt_file))
        self.transform = transform
        self.environment = environment

    def __len__(self):
        return len(self.CMNIST_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.CMNIST_frame.iloc[idx, 0]
        label = self.CMNIST_frame.iloc[idx, 1]
        sample = {'image': image, 'label': label, "environment": self.environment}

        if self.transform:
            sample = {'image': self.transform(image), 'label': label, "environment": self.environment}

        return sample

In [ ]:
cmnist_data = CMNISTDataset(pt_file = "data/ColoredMNIST/train1.pt", environment = 1)

fig = plt.figure()

for i in range(len(cmnist_data)):
    sample = cmnist_data[i]

    #print(i, sample['image'].shape, sample['label'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_CMNIST(**sample)

    if i == 3:
        plt.show()
        break

In [ ]:
cmnist_data1 = CMNISTDataset(pt_file = "data/ColoredMNIST/train1.pt"
                            , environment = 1
                            , transform = transform_to_tensor)
cmnist_data2 = CMNISTDataset(pt_file = "data/ColoredMNIST/train2.pt"
                            , environment = 2
                            , transform = transform_to_tensor)
train_loader = DataLoader(ConcatDataset([cmnist_data1, cmnist_data2]), batch_size=256, num_workers=0)

cmnist_test_data = CMNISTDataset(pt_file = "data/ColoredMNIST/test.pt"
                                 , environment = 3
                                 , transform = transform_to_tensor)
test_loader = DataLoader(cmnist_test_data, batch_size = 512, num_workers = 0)

In [ ]:
def show_batch(sample_batched):
    """Show image with landmarks for a batch of samples."""
    images_batch, landmarks_batch = \
            sample_batched['image'], sample_batched['label']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['label'].size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break

# Copied from exercises and adjusted to our dataset:

In [ ]:
p = Bernoulli(logits=torch.zeros((1000,)))


class ReparameterizedDiagonalGaussian(Distribution):
    """
    A distribution `N(y | mu, sigma I)` compatible with the reparameterization trick given `epsilon ~ N(0, 1)`.
    """
    def __init__(self, mu: Tensor, log_sigma:Tensor):
        assert mu.shape == log_sigma.shape, f"Tensors `mu` : {mu.shape} and ` log_sigma` : {log_sigma.shape} must be of the same shape"
        self.mu = mu
        self.sigma = log_sigma.exp()
        
    #def sample_epsilon(self) -> Tensor:
    #    """`\eps ~ N(0, I)`"""
    #    return torch.empty_like(self.mu).normal_()
        
    #def sample(self) -> Tensor:
    #    """sample `z ~ N(z | mu, sigma)` (without gradients)"""
    #    with torch.no_grad():
    #        return self.rsample()
        
    #def rsample(self) -> Tensor:
    #    """sample `z ~ N(z | mu, sigma)` (with the reparameterization trick) """
    #    self.z = torch.distributions.Normal(self.mu, self.sigma)
    #    return self.z.rsample() # <- your code
        
    def rsample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (with the reparameterization trick) """
        eps = torch.empty_like(self.mu).normal_()
        return self.mu + self.sigma * eps
    # <- your code    
        #return self.mu + self.sigma * self.sample_epsilon() # <- your code    
    
    def log_prob(self, z:Tensor) -> Tensor:
        """return the log probability: log `p(z)`"""
        return - ((z - self.mu)**2)/(2*self.sigma**2) - torch.log(self.sigma) - math.log(math.sqrt(2 * math.pi)) # <- your code
    
    #def log_prob(self, z:Tensor) -> Tensor:
    #    """return the log probability: log `p(z)`"""
    #    dummy = self.rsample()
    #    return self.z.log_prob(z) # <- your code

In [ ]:
#plot a few CMNIST examples
f, axarr = plt.subplots(4, 16, figsize=(16, 4))

# Load a batch of images into memory
sample = next(iter(train_loader))
images = sample['image']
label = sample['label']
environment = sample['environment']

for i, ax in enumerate(axarr.flat):
    ax.imshow(images[i].permute(1,2,0))
    ax.text(1,2,"Label: {}".format(label[i]), backgroundcolor = "white",
             color = "black", fontsize = 6)
    ax.text(1,28,"Environment: {}".format(environment[i]), backgroundcolor = "white",
             color = "black", fontsize = 6)
    ax.axis('off')
    
plt.suptitle('MNIST handwritten digits')
plt.show()

# VAE

In [ ]:
class VariationalAutoencoder(nn.Module):
    """A Variational Autoencoder with
    * a Bernoulli observation model `p_\theta(x | z) = B(x | g_\theta(z))`
    * a Gaussian prior `p(z) = N(z | 0, I)`
    * a Gaussian posterior `q_\phi(z|x) = N(z | \mu(x), \sigma(x))`
    """
    
    def __init__(self, input_shape:torch.Size, latent_features:int) -> None:
        super(VariationalAutoencoder, self).__init__()
        
        self.input_shape = input_shape
        self.latent_features = latent_features
        self.observation_features = np.prod(input_shape)
        
        # define the parameters of the prior, chosen as p(z) = N(0, I)
        ## setting the prior to a vector consisting of zeros with dimensions (1,2*latent_features)
        self.register_buffer('prior_params', torch.zeros(torch.Size([1, 2*latent_features])))
        
         #Inference Network
         #Encode the observation `x` into the parameters of the posterior distribution
         #`q_\phi(z|x) = N(z | \mu(x), \sigma(x)), \mu(x),\log\sigma(x) = h_\phi(x)`
        self.encoderCNN1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.encoderCNN2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.encoderCNN3 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        
        self.encoderFFN2 = nn.Linear(in_features = 32 * 4 * 4, out_features=32 * 4 * 4)
        self.encoderFFN = nn.Linear(in_features = 32 * 4 * 4, out_features=2*latent_features)
        
        #Original network (baseline comparison)
        #self.encoderFFN2 = nn.Linear(in_features = self.observation_features, out_features=256)
        #self.encoderFFN3 = nn.Linear(in_features = 256, out_features=128)
        #self.encoderFFN = nn.Linear(in_features = 128, out_features=2*latent_features)
        
    
         #Generative Model
         #Decode the latent sample `z` into the parameters of the observation model
         #`p_\theta(x | z) = \prod_i B(x_i | g_\theta(x))`
        
        self.relu = nn.ReLU()
        
        #Original network (baseline comparison)
        #self.decoderFFN = nn.Linear(in_features=latent_features, out_features = 128)
        #self.decoderFFN3 = nn.Linear(in_features=128, out_features = 256)
        #self.decoderFFN2 = nn.Linear(in_features = 256, out_features = self.observation_features)
        
        self.decoderFFN = nn.Linear(in_features=latent_features, out_features = 32 * 4 * 4)
        self.decoderFFN2 = nn.Linear(in_features = 32 * 4 * 4, out_features = 32 * 4 * 4)
        
        self.decoderCNN1 = nn.ConvTranspose2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 0)
        self.decoderCNN2 = nn.ConvTranspose2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 1)
        self.decoderCNN3 = nn.ConvTranspose2d(in_channels = 32, out_channels = 3, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 1)
        
    #Define encoder and decoder functions which can be modified to include both a CNN and an ordinary FFN    
    def encoder(self, x):
        
        #Add CNN encoder and flatten x
        
        x = relu(self.encoderCNN1(x))
        x = relu(self.encoderCNN2(x))
        x = relu(self.encoderCNN3(x))
        
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.encoderFFN2(x))
        #x = self.relu(self.encoderFFN3(x)) #Only for running "Original"
        x = self.encoderFFN(x)
        return x
   
    def decoder(self, z):
        
        x = self.relu(self.decoderFFN(z))
        #x = self.relu(self.decoderFFN3(x)) #Only for running "Original"
        x = self.relu(self.decoderFFN2(x))
        
        # reshape x and add CNN decoder
        x = x.view(-1, 32, 4, 4)
        
        x = relu(self.decoderCNN1(x))
        x = relu(self.decoderCNN2(x))
        x = self.decoderCNN3(x)
        return x
        
    def posterior(self, x:Tensor) -> Distribution:
        """return the distribution `q(x|x) = N(z | \mu(x), \sigma(x))`"""
        
        # compute the parameters of the posterior
        h_x = self.encoder(x)
        mu, log_sigma =  h_x.chunk(2, dim=-1)
        
        # return a distribution `q(x|x) = N(z | \mu(x), \sigma(x))`
        return ReparameterizedDiagonalGaussian(mu, log_sigma)
    
    def prior(self, batch_size:int=1)-> Distribution:
        """return the distribution `p(z)`"""
        #Expand prior_params til at være samme antal rækker som i den valgte batch size således at der fås
        #en tensor med dimensionerne (batch_size, 2*latent_features), som så kan udfyldes med
        prior_params = self.prior_params.expand(batch_size, *self.prior_params.shape[-1:])
        #chunk opdeler prior_params i to dele, de første 0-latent_features kollonner indeholder mu og 
        #de sidste n_latent_features inde holder sigmaerne. Nu er der to tensors, som begge har dim
        #(batch_size, n_latent_features). Værdierne i disse tensors, kan bruges til at sample hvordan
        #latent space ser ud (Den der hedder 'latent interpolations' i plots i bunden.)
        mu, log_sigma = prior_params.chunk(2, dim=-1)
        
        # return the distribution `p(z)`
        #BEMÆRK at at det er log_sigma, dvs. at når den inputtes i ReparameterizedDiagonalGaussian så fås mu = 0, sigma = 1
        return ReparameterizedDiagonalGaussian(mu, log_sigma)
    
    def observation_model(self, z:Tensor) -> Distribution:
        """return the distribution `p(x|z)`"""
        px_logits = self.decoder(z)
        px_logits = px_logits.view(-1, *self.input_shape) # reshape the output #old
        #sandsynlighedsfordeling der giver 1 eller 0, baseret på log-odds givet i logits input fra p(x|z).
        #Dvs. at px_logits angiver sandsynligheden for at det givne pixel er henholdsvist rød,grøn,blå. Pixel værdien
        #er enten 0 eller 1. Når man sampler fra bernoulli fordelingen fås dermed et billede, som givet z, giver en figur,
        #som er bestemt af de sandsynligheder der er i px_logits (p(x|z)). Dvs. at for et givet latents space, kan en
        #figur/et tal reproduceres ud fra de beregnede sandsynligheder og den efterfølgende sample fra Bernoulli fordelingen.
        return Bernoulli(logits=px_logits, validate_args=False)
        

    def forward(self, x) -> Dict[str, Any]:
        """compute the posterior q(z|x) (encoder), sample z~q(z|x) and return the distribution p(x|z) (decoder)"""
        
        ###############################################################################################
        # flatten the input
        #x = x.view(x.size(0), -1) #outcommented as part of adding CNN
        
        #### Run through ENCODER and calculate mu and sigma for latent space sampling
        # define the posterior q(z|x) / encode x into q(z|x)
        qz = self.posterior(x)
        
        # sample the posterior using the reparameterization trick: z ~ q(z | x)
        #### LATENT SPACE
        z = qz.rsample()
        
        #### DECODER
        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)
        ###############################################################################################
        
        # define the prior p(z)
        #(Indgår i beregning af kl-term (regularisering) ifm. ELBO) - og bruges også til interpolations visualisering
        #til sidst.
        pz = self.prior(batch_size=x.size(0))
        
        return {'px': px, 'pz': pz, 'qz': qz, 'z': z}
    
    
    def sample_from_prior(self, batch_size:int=100):
        """sample z~p(z) and return p(x|z)"""
        
        # Laver bare reconstruction baseret på latent space
        #Kan evt. fjernes. Anvendes bare til at vise hvor god modellen er til at generere data baseret på
        #latent space genererede data. Funktionen anvendes kun i make_vae_plots.
        
        # degine the prior p(z)
        pz = self.prior(batch_size=batch_size)
        
        # sample the prior 
        z = pz.rsample()
        
        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)
        
        return {'px': px, 'pz': pz, 'z': z}


latent_features = 10 #Husk at opdater denne parameter nede i 'initialization', hvis den skal bruges i VAE loopet også
vae = VariationalAutoencoder(images[0].shape, latent_features)
print(vae)

# ELBO

In [ ]:
def reduce(x:Tensor) -> Tensor:
    """for each datapoint: sum over all dimensions"""
    return x.view(x.size(0), -1).sum(dim=1)

class VariationalInference(nn.Module):
    def __init__(self, beta:float=1.):
        super().__init__()
        self.beta = beta
        
    def forward(self, model:nn.Module, x:Tensor) -> Tuple[Tensor, Dict]:
        
        #The input model in our case is the VAE and the x:tensor is our images.
        
        # forward pass through the model
        outputs = model(x)
        
        # unpack outputs
        px, pz, qz, z = [outputs[k] for k in ["px", "pz", "qz", "z"]]
        
        # evaluate log probabilities
        log_px = reduce(px.log_prob(x)) #log(p(x|z)): Sandsynligheden for at observere vores input variabel x
        #givet vores latent space (tjekker modellens evne til at rekonstruere sig selv, ved at maximere sandsynlig-
        #heden for at observere inputtet selv, givet det konstruerede latent space.
        log_pz = reduce(pz.log_prob(z)) #log(p(z)): Sandsynligheden for at observere vores latent space, givet at
        #latent space følger en standard-normal fordeling (Jo højere sandsynlighed jo bedre)
        log_qz = reduce(qz.log_prob(z)) #log(q(z|x)): Sandsynligheden for at generere netop dette latent space givet 
        #vores input billede x. Denne værdi skal helst være lav?
        
        # compute the ELBO with and without the beta parameter: 
        # `L^\beta = E_q [ log p(x|z) ] - \beta * D_KL(q(z|x) | p(z))`
        # where `D_KL(q(z|x) | p(z)) = log q(z|x) - log p(z)`
        #########################################################################################################
        # Reconstruction loss: E_q [ log p(x|z) ]
        # Regularization term: \beta * D_KL(q(z|x) | p(z))` => Forsøger at tvinge fordelingen q(z|x) mod N(0,1)?
        #########################################################################################################
        kl = log_qz - log_pz
        elbo = log_px - kl
        beta_elbo = log_px - self.beta * kl
        
        # loss
        loss = -beta_elbo.mean()
        
        # prepare the output
        with torch.no_grad():
            diagnostics = {'elbo': elbo, 'log_px':log_px, 'kl': kl}
            
        return loss, diagnostics, outputs
        

In [ ]:
vi = VariationalInference(beta=1)
loss, diagnostics, outputs = vi(vae, images)
print(f"{'loss':6} | mean = {loss:10.3f}, shape: {list(loss.shape)}")
for key, tensor in diagnostics.items():
    print(f"{key:6} | mean = {tensor.mean():10.3f}, shape: {list(tensor.shape)}")

# Initialization

In [ ]:
from collections import defaultdict
# define the models, evaluator and optimizer

# VAE
latent_features = 10 #Hyper parameter
vae = VariationalAutoencoder(images[0].shape, latent_features)

# Evaluator: Variational Inference
beta = 1 #Hyper parameter
vi = VariationalInference(beta=beta)

# The Adam optimizer works really well with VAEs.
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3) #Hyper parameter, tilføj evt. weight_decay (L2 regularization)

# define dictionary to store the training curves
training_data = defaultdict(list)
validation_data = defaultdict(list)

epoch = 0

In [ ]:
torch.manual_seed(1)
num_epochs = 100 #hyper parametre
#batch size hyper parameter can be changed in the dataloader in the beginning.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")

# move the model to the device
vae = vae.to(device)

# training..
while epoch < num_epochs:
    epoch+= 1
    training_epoch_data = defaultdict(list)
    vae.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        loss, diagnostics, outputs = vi(vae, x)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # gather data for the current bach
        for k, v in diagnostics.items():
            training_epoch_data[k] += [v.mean().item()]
            

    # gather data for the full epoch
    for k, v in training_epoch_data.items():
        training_data[k] += [np.mean(training_epoch_data[k])]

    # Evaluate on a single batch, do not propagate gradients
    with torch.no_grad():
        vae.eval()
        
        # Just load a single batch from the test loader
        sample = next(iter(test_loader))
        x = sample['image']
        y = sample['label']
        x = x.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        loss, diagnostics, outputs = vi(vae, x)
        
        # gather data for the validation step
        for k, v in diagnostics.items():
            validation_data[k] += [v.mean().item()]
    
    # Reproduce the figure from the begining of the notebook, plot the training curves and show latent samples
    make_vae_plots(vae, x, y, outputs, training_data, validation_data)

In [ ]:
torch.save(vae.state_dict(), '100epochs_VAE_flipped.pt')

In [ ]:
#vae.load_state_dict(torch.load('100epochs_VAE_flipped.pt', map_location=device))
#vae = vae.to(device)

In [ ]:
qz = vae.posterior(sample['image'])
z = qz.rsample()

# Phase 2 - PC algorithm based on latent variable

(page 6-7)

In [ ]:
#Dummy code for the testing of the PC algorithm to ensure the model data is prepared in the right format
import numpy as np
import pandas as pd
import cdt
cdt.SETTINGS.rpath = 'C:/Program Files/R/R-4.2.2/bin/Rscript' # this path should point to your own R implementation !
from cdt.causality.graph import PC
import networkx as nx
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

device = torch.device("cpu")
print(f">> Using device: {device}")

#names = np.array(["A","B", "C", "D", "E"])
#data_df = pd.DataFrame(data, columns = names)
#pc_test = PC(CItest = 'gaussian', alpha = 0.05, verbose=False).create_graph_from_data(data_df[names[permutation]])

#nx.draw(pc_test, with_labels=True, font_weight='bold')
#plt.show()

### Implementing PC algorithm on CMNIST iVAE result ###
#load full CMNIST dataset --- determine causal latent variables based on training data
# -> assume ALL test data are reserved to after all training is done.
#cmnist_data1 = CMNISTDataset(pt_file = "data/ColoredMNIST/train1.pt"
#                            , environment = 1
#                            , transform = transform_to_tensor)
#cmnist_data2 = CMNISTDataset(pt_file = "data/ColoredMNIST/train2.pt"
#                            , environment = 2
#                           , transform = transform_to_tensor)

#cmnist = ConcatDataset([cmnist_data1, cmnist_data2])
#loader = DataLoader(cmnist, batch_size = 40000, batch_sampler = None)

sample = next(iter(loader))

x = sample['image']
y = sample['label'].reshape(-1,1)
e = sample['environment'].reshape(-1,1)

x = x.to(device)
y = y.to(device)
e = e.to(device)

vae = vae.to(device)

output = vae.forward(x)

z = output['z']

z_full = z
x_full = x
y_full = y
e_full = e

In [ ]:
qz = output['qz']
z_sample1 = qz.rsample()
z_sample2 = qz.rsample()
z_sample3 = qz.rsample()
z_sample4 = qz.rsample()
z_sample5 = qz.rsample()
z_sample6 = qz.rsample()
z_sample7 = qz.rsample()
z_sample8 = qz.rsample()
z_sample9 = qz.rsample()
z_sample10 = qz.rsample()

y = y.detach()
e = e.detach()



y_df = pd.DataFrame(torch.cat((y,y,y,y,y,y,y,y,y,y), dim = 0))
z_df = pd.DataFrame(torch.cat((z_sample1.detach(),z_sample2.detach(),z_sample3.detach(),z_sample4.detach(),z_sample5.detach(),
    z_sample6.detach(),z_sample7.detach(),z_sample8.detach(),z_sample9.detach(),z_sample10.detach(),), dim = 0))
df = y_df#; df = z_df; #df = z_df;  
df['Z1'] = z_df[0]
df['Z2'] = z_df[1]
df['Z3'] = z_df[2]
df['Z4'] = z_df[3]
df['Z5'] = z_df[4]
df['Z6'] = z_df[5]
df['Z7'] = z_df[6]
df['Z8'] = z_df[7]
df['Z9'] = z_df[8]
df['Z10'] = z_df[9]


df.columns = ['Y','Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8', 'Z9', 'Z10']
df.head()

df.shape

In [ ]:
glasso = cdt.independence.graph.Glasso()

skeleton = glasso.predict(df)

model_pc = cdt.causality.graph.PC(alpha =0.05)

from causallearn.search.ConstraintBased.PC  import pc

from causallearn.utils.cit import fisherz
cg = pc(np.array(df), 0.1, kernelZ='Polynomial', approx=False, est_width='median')

cg.draw_pydot_graph()
#graph_pc = model_pc.predict(df, skeleton)

#nx.draw(cg, with_labels=True, font_weight='bold', font_size = 8)
#plt.savefig('PC_iVAE_alpha_005.png', transparent=True)
#plt.show()

In [ ]:
pc = PC(alpha=0.05)
pc_output = pc.predict(df)

nx.draw(pc_output, with_labels=True, font_weight='bold', font_size = 16, node_size = 9000)
plt.savefig('PC_iVAE_alpha_005.png', transparent=True)
plt.show()

In [ ]:
def find_parents(pc_array,y_pos,z_pos):
    ind = np.where(pc_array[z_pos,y_pos] == 1)
    ind_DAGpar = []
    ind_par = []
    for i in ind[0]:
        if pc_array[y_pos,i] == 0:
            ind_DAGpar.append(i)
        else:
            ind_par.append(i)
    return ind_DAGpar, ind_par

pc = PC(alpha=0.01)
pc_output = pc.predict(df)

nx.draw(pc_output, with_labels=True, font_weight='bold', font_size = 16, node_size = 9000)
plt.savefig('PC_iVAE_alpha_005.png', transparent=True)
plt.show()

pc_array = pc._run_pc(df)
parents = torch.tensor(find_parents(pc_array, 0, range(0,10))[0]).to(device)
print(parents)

In [ ]:
parents = torch.tensor(find_parents(pc_array, 0, range(0,10))[0]).to(device)
parents = parents - 1 #For at følge rækkefølgen fra z
print(parents)

In [ ]:
#Found indices:

#tensor([0, 3, 4, 6]) kører, og har alpha = 0.05 - Skal testes, men umiddelbart dårligt
#tensor([3, 4, 6]) 
#parents = parents.to("cpu")
parents = torch.tensor([0,4,9])
options = np.array([0,1,2,3,4,5,6,7,8,9])
children = np.delete(options, parents)
nParents = len(parents)
nChildren = 10 - nParents

In [ ]:
#Accuracy function from week 4:
def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

#Building a classifier:

class ClassifierVAE(nn.Module):
    def __init__(self):
        super().__init__()

        #Layers
        self.input_layer = nn.Linear(in_features=3, out_features = 50)
        self.layer1 = nn.Linear(in_features = 50, out_features = 100)
        self.output_layer = nn.Linear(in_features = 100, out_features = 1)

        #activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    #forward function(self, x):
    def forward(self, x):
        
        x = self.input_layer(x)
        x = self.relu(self.layer1(x))
        x = self.sigmoid(self.output_layer(x))

        return x
    # 

classifierVAE = ClassifierVAE()
classifierVAE.to(device)



In [ ]:
import torch.optim as optim
from sklearn import metrics

#Define loss function
loss_fn = nn.BCELoss() 
#The adam algorithm automatically use momentum
optimizer = optim.Adam(classifierVAE.parameters(), lr = 1e-4) 

#Test model
#out = classifierVAE(z[1])
#print("Output shape:", out.size())
#print(f"Output logits:\n{out.detach().cpu().numpy()}")

In [ ]:
#TRAINING THE CLASSIFIER
epoch = 0
num_epochs = 10 #hyper parametre
#batch size hyper parameter can be changed in the dataloader in the beginning.

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f">> Using device: {device}")

vae = vae.to(device)
classifierVAE = classifierVAE.to(device)
parents = parents.to(device)

# training..
training_epoch_data = []
test_epoch_data = []


# training..
training_epoch_data = []
test_epoch_data = []
test_acc, train_acc = [], []

while epoch < num_epochs:
    epoch+= 1
    training_batch_data = []
    test_batch_data = []
    classifierVAE.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label'].to(torch.float32)
        y = y.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = vae.posterior(x)
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        outputVAE = classifierVAE(z).view(-1)

        loss = loss_fn(outputVAE, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # gather data for the current batch
        training_batch_data.append(loss)
            

    # gather data for the full epoch
    training_epoch_data.append(torch.tensor(training_batch_data).mean())

    print("Epoch: {}, Mean epoch loss: {}".format(epoch, torch.tensor(training_batch_data).mean()))
    

In [ ]:
#NF_ivae = NF_ivae.to(device)
#indices = indices.to(device)
#parents = parents.to(device)

#Set iVAE to eval - the no.grad thing is not an issue as we are only using the encoder.
#NF_ivae.eval()

#TRAINING THE CLASSIFIER
epoch = 0
#batch size hyper parameter can be changed in the dataloader in the beginning.

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f">> Using device: {device}")

# training..
training_epoch_data = []
test_epoch_data = []
test_acc, train_acc = [], []


#Evaluate training
with torch.no_grad():
    classifierVAE.eval()
    train_targs, train_preds = [], []
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label']
        y = y.to(device)

        #e = sample['environment']
        #e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = vae.posterior(x)
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifierVAE(z).view(-1)
        prediction = torch.round(output)

        train_targs += list(y.cpu().numpy())
        train_preds += list(prediction.data.cpu().numpy())

    #Evaluating validation
    val_targs, val_preds = [], []
    for sample in test_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label']
        y = y.to(device)

        #e = sample['environment']
        #e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = vae.posterior(x)
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifierVAE(z).view(-1)
        prediction = torch.round(output)


        val_targs += list(y.cpu().numpy())
        val_preds += list(prediction.data.cpu().numpy())

train_acc_cur = accuracy(Tensor(train_targs),Tensor(train_preds))
train_acc.append(train_acc_cur)

test_acc_cur = accuracy(Tensor(val_targs),Tensor(val_preds))
test_acc.append(test_acc_cur)
print("Epoch %2i : , Train acc %f, Valid acc %f" % (
            epoch, train_acc_cur, test_acc_cur))



In [ ]:
torch.save(classifierVAE.state_dict(), 'classifierVAEv1.pt')

# iVAE

In [ ]:
class iVariationalAutoencoder(nn.Module):
    """A Variational Autoencoder with
    * a Bernoulli observation model `p_\theta(x | z) = B(x | g_\theta(z))`
    * a Gaussian prior `p(z) = N(z | 0, I)`
    * a Gaussian posterior `q_\phi(z|x) = N(z | \mu(x), \sigma(x))`
    """
    
    def __init__(self, input_shape:torch.Size, latent_features:int, device) -> None:
        super(iVariationalAutoencoder, self).__init__()
        
        self.input_shape = input_shape
        self.latent_features = latent_features

        self.device = device

        # define the parameters of the prior, chosen as p(z) = N(0, I)
        ## setting the prior to a vector consisting of zeros with dimensions (1,2*latent_features)
        # self.register_buffer('prior_params', torch.zeros(torch.Size([1, 2*latent_features])))
        
        '''
        According to page 31-32 the iVAE consist of 7 NNs:
        1. lambdaf prior
        
        2. X-encoder (Classic image CNN)
        3. (Y, E)-encoder
        4. (X, Y, E)-merger/encoder

        5. Decoder

        1: Learn priors based on the label distribution for the given environment
        2-4: Encoding X, encoding Y and E and merging these two encoders, to generate a 
             qz which is conditional on the environment.
        5: Decodes the latent space through pz. Since the latent space now contain some measure
           of environment, then this distribution pz is consequentially conditioned on the environment

        NN 1-3 can be found in the variational inference funktion.
        '''
        #### PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS ####
        #NN 1/5
        self.Lambdaf_prior = nn.Sequential(
            nn.Linear(in_features = 2, out_features=50), #Input
            nn.Linear(in_features = 50, out_features=50), #Fully connected
            nn.ReLU(),
            nn.Linear(in_features = 50, out_features = 20))  #Output
        #### PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS ####

         #For NN 2/5 X-Encoder: Inference Network
         #Encode the observation `x` into the parameters of the posterior distribution
         #`q_\phi(z|x) = N(z | \mu(x), \sigma(x)), \mu(x),\log\sigma(x) = h_\phi(x)`
        self.encoderCNN1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.encoderCNN2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.encoderCNN3 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        ##NN 3/5 (Y, E)-Encoder
        self.YEencoder = nn.Sequential(
            nn.Linear(in_features = 2, out_features=100),
            nn.Linear(in_features = 100, out_features = 100),
            nn.ReLU())

        ##NN 4/5 (X, Y, E)-merger/encoder
        #remember to concatenate x.flatten, y, e before running this.
        self.XYEmerger = nn.Sequential(
            nn.Linear(in_features = 32 * 4 * 4 + 100, out_features=100),
            nn.Linear(in_features = 100, out_features = 100),
            nn.ReLU(),
            nn.Linear(in_features = 100, out_features = 2*latent_features))


        #For NN 5/5 (Decoder): Generative Model
        #Decode the latent sample `z` into the parameters of the observation model
        #`p_\theta(x | z) = \prod_i B(x_i | g_\theta(x))`
        self.decoderFFN = nn.Linear(in_features=latent_features, out_features = 32 * 4 * 4)
        self.decoderFFN2 = nn.Linear(in_features = 32 * 4 * 4, out_features = 32 * 4 * 4)
        
        self.decoderCNN1 = nn.ConvTranspose2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 0)
        self.decoderCNN2 = nn.ConvTranspose2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 1)
        self.decoderCNN3 = nn.ConvTranspose2d(in_channels = 32, out_channels = 3, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 1)

    def prior_params(self, y, e):
        """return the distribution `p(z)`"""
        ye = torch.cat((y, e), dim = 1)
        ye = ye.to(torch.float32)
        lambdaf_parameters = self.Lambdaf_prior(ye)

        return lambdaf_parameters

    #NN 4/7
    def encoder(self, x, y, e):
        x = self.relu(self.encoderCNN1(x))
        x = self.relu(self.encoderCNN2(x))
        x = self.relu(self.encoderCNN3(x))
        x = x.view(x.size(0), -1) #NN 4/7

        ye = torch.cat((y, e), dim = 1)
        ye = ye.to(torch.float32)
        ye = self.YEencoder(ye) #NN 5/7

        xye = torch.cat((x,ye), dim = 1)
        xye = self.XYEmerger(xye) #NN 6/7
    
        return xye
   
    #NN 7/7
    def decoder(self, z):
        
        x = self.relu(self.decoderFFN(z))
        x = self.relu(self.decoderFFN2(x))
        
        # reshape x and add CNN decoder
        x = x.view(-1, 32, 4, 4)
        
        x = self.relu(self.decoderCNN1(x))
        x = self.relu(self.decoderCNN2(x))
        x = self.decoderCNN3(x)
        return x    
        
    def posterior(self, x:Tensor, y:Tensor, z:Tensor) -> Distribution:
        """return the distribution `q(x|x) = N(z | \mu(x), \sigma(x))`"""
        
        # compute the parameters of the posterior
        h_x = self.encoder(x, y, z)
        mu, log_sigma =  h_x.chunk(2, dim=-1)
        
        # return a distribution `q(x|x) = N(z | \mu(x), \sigma(x))`
        return ReparameterizedDiagonalGaussian(mu, log_sigma)
    
    def prior(self, y, e)-> Distribution:
        """return the distribution `p(z)`"""
        #Expand prior_params til at være samme antal rækker som i den valgte batch size således at der fås
        #en tensor med dimensionerne (batch_size, 2*latent_features), som så kan udfyldes med
        prior_params = self.prior_params(y, e)
        #chunk opdeler prior_params i to dele, de første 0-latent_features kollonner indeholder mu og 
        #de sidste n_latent_features inde holder sigmaerne. Nu er der to tensors, som begge har dim
        #(batch_size, n_latent_features). Værdierne i disse tensors, kan bruges til at sample hvordan
        #latent space ser ud (Den der hedder 'latent interpolations' i plots i bunden.)
        mu, log_sigma = prior_params.chunk(2, dim=-1)
        
        # return the distribution `p(z)`
        #BEMÆRK at at det er log_sigma, dvs. at når den inputtes i ReparameterizedDiagonalGaussian så fås mu = 0, sigma = 1
        return ReparameterizedDiagonalGaussian(mu, log_sigma)
    
    def observation_model(self, z:Tensor) -> Distribution:
        """return the distribution `p(x|z)`"""
        px_means = self.decoder(z)
        px_means = px_means.view(-1, *self.input_shape) # reshape the output #old
        log_var = 0.0025 * torch.ones(px_means.shape)
        log_sigma = torch.log(torch.sqrt(log_var))
        log_sigma = log_sigma.to(self.device)
        #sandsynlighedsfordeling der giver 1 eller 0, baseret på log-odds givet i logits input fra p(x|z).
        #Dvs. at px_logits angiver sandsynligheden for at det givne pixel er henholdsvist rød,grøn,blå. Pixel værdien
        #er enten 0 eller 1. Når man sampler fra bernoulli fordelingen fås dermed et billede, som givet z, giver en figur,
        #som er bestemt af de sandsynligheder der er i px_logits (p(x|z)). Dvs. at for et givet latents space, kan en
        #figur/et tal reproduceres ud fra de beregnede sandsynligheder og den efterfølgende sample fra Bernoulli fordelingen.
        return ReparameterizedDiagonalGaussian(mu = px_means, log_sigma = log_sigma)
        

    def forward(self, x, y, e) -> Dict[str, Any]:
        """compute the posterior q(z|x) (encoder), sample z~q(z|x) and return the distribution p(x|z) (decoder)"""
        
        ###############################################################################################
        # flatten the input
        #x = x.view(x.size(0), -1) #outcommented as part of adding CNN
        
        #### Run through ENCODER and calculate mu and sigma for latent space sampling
        # define the posterior q(z|x) / encode x into q(z|x)
        qz = self.posterior(x, y, e)
        
        # sample the posterior using the reparameterization trick: z ~ q(z | x)
        #### LATENT SPACE
        z = qz.rsample()
        
        #### DECODER
        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)
        ###############################################################################################
        
        # define the prior p(z)
        #(Indgår i beregning af kl-term (regularisering) ifm. ELBO) - og bruges også til interpolations visualisering
        #til sidst.
        pz = self.prior(y,e)
        
        return {'px': px, 'pz': pz, 'qz': qz, 'z': z}
    
    
    def sample_from_prior(self, y, e):
        """sample z~p(z) and return p(x|z)"""
        
        # Laver bare reconstruction baseret på latent space
        #Kan evt. fjernes. Anvendes bare til at vise hvor god modellen er til at generere data baseret på
        #latent space genererede data. Funktionen anvendes kun i make_vae_plots.
        
        # degine the prior p(z)
        pz = self.prior(y, e)
        
        # sample the prior 
        z = pz.rsample()
        
        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)
        
        return {'px': px, 'pz': pz, 'z': z}
    
    def reduce(self, x:Tensor) -> Tensor:
        """for each datapoint: sum over all dimensions"""
        return x.view(x.size(0), -1).sum(dim=1)

    def VariationalInference(self, x, y, e, beta):
        self.beta = beta
        # forward pass through the model
        outputs = self.forward(x, y, e)
        
        # unpack outputs
        px, pz, qz, z = [outputs[k] for k in ["px", "pz", "qz", "z"]]
        
        # evaluate log probabilities
        log_px = self.reduce(px.log_prob(x)) #log(p(x|z)): Sandsynligheden for at observere vores input variabel x
        #givet vores latent space (tjekker modellens evne til at rekonstruere sig selv, ved at maximere sandsynlig-
        #heden for at observere inputtet selv, givet det konstruerede latent space.
        log_pz = self.reduce(pz.log_prob(z)) #log(p(z)): Sandsynligheden for at observere vores latent space, givet at
        #latent space følger en standard-normal fordeling (Jo højere sandsynlighed jo bedre)
        log_qz = self.reduce(qz.log_prob(z)) #log(q(z|x)): Sandsynligheden for at generere netop dette latent space givet 
        #vores input billede x. Denne værdi skal helst være lav?
        
        # compute the ELBO with and without the beta parameter: 
        # `L^\beta = E_q [ log p(x|z) ] - \beta * D_KL(q(z|x) | p(z))`
        # where `D_KL(q(z|x) | p(z)) = log q(z|x) - log p(z)`
        #########################################################################################################
        # Reconstruction loss: E_q [ log p(x|z) ]
        # Regularization term: \beta * D_KL(q(z|x) | p(z))` => Forsøger at tvinge fordelingen q(z|x) mod N(0,1)?
        #########################################################################################################
        kl = log_qz - log_pz
        elbo = log_px - kl
        beta_elbo = log_px - self.beta * kl
        
        # loss
        loss = -beta_elbo.mean()
        
        # prepare the output
        with torch.no_grad():
            diagnostics = {'elbo': elbo, 'log_px':log_px, 'kl': kl}
            
        return loss, diagnostics, outputs

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f">> Using device: {device}")

latent_features = 10 #Husk at opdater denne parameter nede i 'initialization', hvis den skal bruges i VAE loopet også
ivae = iVariationalAutoencoder(images[0].shape, latent_features, device)
ivae = ivae.to(device)
print(ivae)

In [ ]:
ivae.load_state_dict(torch.load('100epochs_iVAE_var005_flipped.pt', map_location=device))

In [ ]:
#Dummy code for the testing of the PC algorithm to ensure the model data is prepared in the right format
import numpy as np
import pandas as pd
import cdt
cdt.SETTINGS.rpath = 'C:/Program Files/R/R-4.2.2/bin/Rscript' # this path should point to your own R implementation !
from cdt.causality.graph import PC
import networkx as nx
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

device = torch.device("cpu")
print(f">> Using device: {device}")

#cmnist = ConcatDataset([cmnist_data1, cmnist_data2])
#loader = DataLoader(cmnist, batch_size = 40000, batch_sampler = None)

sample = next(iter(loader))

x = sample['image']
y = sample['label'].reshape(-1,1)
e = sample['environment'].reshape(-1,1)

x = x.to(device)
y = y.to(device)
e = e.to(device)

ivae = ivae.to(device)

output = ivae.forward(x, y, e)

z = output['z']

z_full = z
x_full = x
y_full = y
e_full = e

In [ ]:
qz = output['qz']
z_sample1 = qz.rsample()
z_sample2 = qz.rsample()
z_sample3 = qz.rsample()
z_sample4 = qz.rsample()
z_sample5 = qz.rsample()
z_sample6 = qz.rsample()
z_sample7 = qz.rsample()
z_sample8 = qz.rsample()
z_sample9 = qz.rsample()
z_sample10 = qz.rsample()

y = y.detach()
e = e.detach()



y_df = pd.DataFrame(torch.cat((y,y,y,y,y,y,y,y,y,y), dim = 0))
z_df = pd.DataFrame(torch.cat((z_sample1.detach(),z_sample2.detach(),z_sample3.detach(),z_sample4.detach(),z_sample5.detach(),
    z_sample6.detach(),z_sample7.detach(),z_sample8.detach(),z_sample9.detach(),z_sample10.detach(),), dim = 0))
df = y_df#; df = z_df; #df = z_df;  
df['Z1'] = z_df[0]
df['Z2'] = z_df[1]
df['Z3'] = z_df[2]
df['Z4'] = z_df[3]
df['Z5'] = z_df[4]
df['Z6'] = z_df[5]
df['Z7'] = z_df[6]
df['Z8'] = z_df[7]
df['Z9'] = z_df[8]
df['Z10'] = z_df[9]


df.columns = ['Y','Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8', 'Z9', 'Z10']
df.head()

df.shape

In [ ]:
def find_parents(pc_array,y_pos,z_pos):
    ind = np.where(pc_array[z_pos,y_pos] == 1)
    ind_DAGpar = []
    ind_par = []
    for i in ind[0]:
        if pc_array[y_pos,i] == 0:
            ind_DAGpar.append(i)
        else:
            ind_par.append(i)
    return ind_DAGpar, ind_par

pc = PC(alpha=0.001)
pc_output = pc.predict(df)

nx.draw(pc_output, with_labels=True, font_weight='bold', font_size = 16, node_size = 9000)
plt.savefig('PC_iVAE_alpha_005.png', transparent=True)
plt.show()

pc_array = pc._run_pc(df)
parents = torch.tensor(find_parents(pc_array, 0, range(0,10))[0]).to(device)
parents = parents - 1
print(parents)

In [ ]:
#Found indices:

#tensor([0, 3, 4, 6]) kører, og har alpha = 0.05 - Skal testes, men umiddelbart dårligt
#tensor([2, 5]) kører, cherry-picked. Har dårlig performance på iVAE
#tensor([3, 4, 6]) 
#parents = parents.to("cpu")
parents = torch.tensor([2,7])
options = np.array([0,1,2,3,4,5,6,7,8,9])
children = np.delete(options, parents)
nParents = len(parents)
nChildren = 10 - nParents

In [ ]:
#Accuracy function from week 4:
def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

#Building a classifier:

class ClassifieriVAE(nn.Module):
    def __init__(self, indices):
        super().__init__()
        self.indices = indices
        #Layers
        self.input_layer = nn.Linear(in_features=2, out_features = 50)
        self.layer1 = nn.Linear(in_features = 50, out_features = 100)
        self.output_layer = nn.Linear(in_features = 100, out_features = 1)

        #activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid() 
    #forward function(self, x):
    def forward(self, x):
        
        x = self.input_layer(x)
        x = self.relu(self.layer1(x))
        x = self.sigmoid(self.output_layer(x))

        return x
    # 

classifieriVAE = ClassifieriVAE(parents)
classifieriVAE = classifieriVAE.to(device)

In [ ]:
import torch.optim as optim
from sklearn import metrics

#Define loss function
loss_fn = nn.BCELoss() 
#The adam algorithm automatically use momentum
optimizer = optim.Adam(classifieriVAE.parameters(), lr = 1e-4) 

#Test model
#z = output['z']
#z = torch.index_select(z, 1, parents)
#out = classifier(z[1])
#print("Output shape:", out.size())
#print(f"Output logits:\n{out.detach().cpu().numpy()}")

In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")

ivae = ivae.to(device)
z = z.to(device)
parents = parents.to(device)
classifieriVAE = classifieriVAE.to(device)

#Set iVAE to eval - the no.grad thing is not an issue as we are only using the encoder.
ivae.eval()

#TRAINING THE CLASSIFIER
epoch = 0
num_epochs = 10 #hyper parametre
#batch size hyper parameter can be changed in the dataloader in the beginning.
#device = torch.device("cpu")

# training..
training_epoch_data = []
test_epoch_data = []
test_acc, train_acc = [], []

while epoch < num_epochs:
    epoch+= 1
    training_batch_data = []
    test_batch_data = []
    classifieriVAE.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label'].to(torch.float32)
        y = y.to(device)

        e = sample['environment'].to(torch.float32)
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = ivae.posterior(x, y.reshape(-1,1), e.reshape(-1,1))
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifieriVAE(z).view(-1)

        loss = loss_fn(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # gather data for the current batch
        training_batch_data.append(loss)
            

    # gather data for the full epoch
    training_epoch_data.append(torch.tensor(training_batch_data).mean())
    print("Epoch: {}, Mean epoch loss: {}".format(epoch, torch.tensor(training_batch_data).mean()))
    #print("Epoch {} : , Loss {}".format(epoch, training_epoch_data[epoch-1]))


In [ ]:
#NF_ivae = NF_ivae.to(device)
#indices = indices.to(device)
#parents = parents.to(device)

#Set iVAE to eval - the no.grad thing is not an issue as we are only using the encoder.
#NF_ivae.eval()

#TRAINING THE CLASSIFIER
epoch = 0
#batch size hyper parameter can be changed in the dataloader in the beginning.

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f">> Using device: {device}")

# training..
training_epoch_data = []
test_epoch_data = []
test_acc, train_acc = [], []


#Evaluate training
with torch.no_grad():
    classifieriVAE.eval()
    train_targs, train_preds = [], []
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label']
        y = y.to(device)

        e = sample['environment']
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = ivae.posterior(x, y.reshape(-1,1), e.reshape(-1,1))
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifieriVAE(z).view(-1)
        prediction = torch.round(output)

        train_targs += list(y.cpu().numpy())
        train_preds += list(prediction.data.cpu().numpy())

    #Evaluating validation
    val_targs, val_preds = [], []
    for sample in test_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label']
        y = y.to(device)

        e = sample['environment']
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = ivae.posterior(x, y.reshape(-1,1), e.reshape(-1,1))
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifieriVAE(z).view(-1)
        prediction = torch.round(output)


        val_targs += list(y.cpu().numpy())
        val_preds += list(prediction.data.cpu().numpy())

train_acc_cur = accuracy(Tensor(train_targs),Tensor(train_preds))
train_acc.append(train_acc_cur)

test_acc_cur = accuracy(Tensor(val_targs),Tensor(val_preds))
test_acc.append(test_acc_cur)
print("Epoch %2i : , Train acc %f, Valid acc %f" % (
            epoch, train_acc_cur, test_acc_cur))



In [ ]:
#torch.save(classifieriVAE.state_dict(), 'classifieriVAEv1.pt')

In [ ]:
pd.DataFrame([np.array(val_targs).mean(),np.array(val_preds).mean()]).T

In [ ]:
print("Performance compared to iCaRL in the paper (table on page. 27): {}%".format(round(correct/(correct+false) * 100 / 68.75 * 100,2)))

print("Performance: {}%".format(round(correct/(correct+false) * 100,2)))

# NF-iVAE

In [ ]:
from torch.distributions import Bernoulli, Normal
def reduce(x:Tensor) -> Tensor:
    """for each datapoint: sum over all dimensions"""
    return x.view(x.size(0), -1).sum(dim=1)

class NF_iVAE(nn.Module):
    """A Variational Autoencoder with
    * a Bernoulli observation model `p_\theta(x | z) = B(x | g_\theta(z))`
    * a Gaussian prior `p(z) = N(z | 0, I)`
    * a Gaussian posterior `q_\phi(z|x) = N(z | \mu(x), \sigma(x))`
    """
    
    def __init__(self, input_shape:torch.Size, latent_features:int, device) -> None:
        super(NF_iVAE, self).__init__()
        
        self.input_shape = input_shape
        self.latent_features = latent_features

        self.device = device#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #self.device = torch.device("cpu")
        # define the parameters of the prior, chosen as p(z) = N(0, I)
        ## setting the prior to a vector consisting of zeros with dimensions (1,2*latent_features)
        # self.register_buffer('prior_params', torch.zeros(torch.Size([1, 2*latent_features])))
        
        '''
        According to page 31-32 the iVAE consist of 7 NNs:
        1. TNN prior
        2. lambdaNN prior
        3. lambdaf prior
        
        4. X-encoder (Classic image CNN)
        5. (Y, E)-encoder
        6. (X, Y, E)-merger/encoder

        7. Decoder

        1-3: Learn priors based on the label distribution for the given environment
        4-6: Encoding X, encoding Y and E and merging these two encoders, to generate a 
             qz which is conditional on the environment.
        7: Decodes the latent space through pz. Since the latent space now contain some measure
           of environment, then this distribution pz is consequentially conditioned on the environment

        NN 1-3 can be found in the variational inference funktion.
        '''
        #### PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS ####
        #NN 1/7
        self.TNN_prior = nn.Sequential(
            nn.Linear(in_features = latent_features, out_features=50), #Input
            nn.Linear(in_features = 50, out_features=50), #Fully connected
            nn.ReLU(),
            nn.Linear(in_features = 50, out_features = 45)) #Output

        #NN 2/7
        self.LambdaNN_prior = nn.Sequential(
            nn.Linear(in_features = 2, out_features=50), #Input
            nn.Linear(in_features = 50, out_features=50), #Fully connected
            nn.ReLU(),
            nn.Linear(in_features = 50, out_features = 45)) #Output

        #NN 3/7
        self.Lambdaf_prior = nn.Sequential(
            nn.Linear(in_features = 2, out_features=50), #Input
            nn.Linear(in_features = 50, out_features=50), #Fully connected
            nn.ReLU(),
            nn.Linear(in_features = 50, out_features = 20))  #Output
        #### PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS PRIORS ####

         #For NN 4/7 X-Encoder: Inference Network
         #Encode the observation `x` into the parameters of the posterior distribution
         #`q_\phi(z|x) = N(z | \mu(x), \sigma(x)), \mu(x),\log\sigma(x) = h_\phi(x)`
        self.encoderCNN1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.encoderCNN2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.encoderCNN3 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        ##NN 5/7 (Y, E)-Encoder
        self.YEencoder = nn.Sequential(
            nn.Linear(in_features = 2, out_features=100),
            nn.Linear(in_features = 100, out_features = 100),
            nn.ReLU())

        ##NN 6/7 (X, Y, E)-merger/encoder
        #remember to concatenate x.flatten, y, e before running this.
        self.XYEmerger = nn.Sequential(
            nn.Linear(in_features = 32 * 4 * 4 + 100, out_features=100),
            nn.Linear(in_features = 100, out_features = 100),
            nn.ReLU(),
            nn.Linear(in_features = 100, out_features = 2*latent_features))


        #For NN 7/7 (Decoder): Generative Model
        #Decode the latent sample `z` into the parameters of the observation model
        #`p_\theta(x | z) = \prod_i B(x_i | g_\theta(x))`
        self.decoderFFN = nn.Linear(in_features=latent_features, out_features = 32 * 4 * 4)
        self.decoderFFN2 = nn.Linear(in_features = 32 * 4 * 4, out_features = 32 * 4 * 4)
        
        self.decoderCNN1 = nn.ConvTranspose2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 0)
        self.decoderCNN2 = nn.ConvTranspose2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 1)
        self.decoderCNN3 = nn.ConvTranspose2d(in_channels = 32, out_channels = 3, kernel_size = 3, stride = 2, padding = 1
                              ,output_padding = 1)

    #Non-factorized prior for NF-iVAE from page 28
    def non_factorized_prior_pz(self, z, TNN_parameters, lambdaNN_parameters, lambdaf_parameters, reduce = True):
        #beregning fra side 28 under M.2. P_T,lambda(Z|Y,E).
        z_cat = torch.cat((z, z.pow(2)), dim = 1) 
        # "(*).sum(dim=1)" is the vector-vector dot product. It is not possible to make this calculation with e.g. torch.tensordot, as we want the dot product of each vector and not the entire batch
        non_factorized_prior = (TNN_parameters*lambdaNN_parameters).sum(dim = 1) + (z_cat * lambdaf_parameters).sum(dim = 1)
        #the f,z term - (z_cat * lambdaf_parameters).sum(dim = 1): Is equivalent to the factorized exponential family
        #the NN term - (TNN_parameters*lambdaNN_parameters).sum(dim = 1): Capture the dependencies between the latent variables
        return non_factorized_prior

    def parameters_for_non_factorized_prior(self, z, y, e):
        """return the distribution `p(z)`"""
        TNN_parameters = self.TNN_prior(z)
        ye = torch.cat((y, e), dim = 1)
        ye = ye.to(torch.float32)
        LambdaNN_parameters = self.LambdaNN_prior(ye)
        lambdaf_parameters = self.Lambdaf_prior(ye)

        return TNN_parameters, LambdaNN_parameters, lambdaf_parameters

    #NN 4/7
    def encoder(self, x, y, e):
        x = self.relu(self.encoderCNN1(x))
        x = self.relu(self.encoderCNN2(x))
        x = self.relu(self.encoderCNN3(x))
        x = x.view(x.size(0), -1) #NN 4/7

        ye = torch.cat((y, e), dim = 1)
        ye = ye.to(torch.float32)
        ye = self.YEencoder(ye) #NN 5/7

        xye = torch.cat((x,ye), dim = 1)
        xye = self.XYEmerger(xye) #NN 6/7
    
        return xye
   
    #NN 7/7
    def decoder(self, z):
        
        x = self.relu(self.decoderFFN(z))
        x = self.relu(self.decoderFFN2(x))
        
        # reshape x and add CNN decoder
        x = x.view(-1, 32, 4, 4)
        
        x = self.relu(self.decoderCNN1(x))
        x = self.relu(self.decoderCNN2(x))
        x = self.decoderCNN3(x)
        return x
        
    def posterior(self, x:Tensor, y:Tensor, e:Tensor) -> Distribution:
        """return the distribution `q(x|x) = N(z | \mu(x), \sigma(x))`"""
        
        # compute the parameters of the posterior
        h_x = self.encoder(x, y, e)
        mu, log_sigma =  h_x.chunk(2, dim=-1)
        
        # return a distribution `q(x|x) = N(z | \mu(x), \sigma(x))`
        return ReparameterizedDiagonalGaussian(mu, log_sigma)

    
    def observation_model(self, z:Tensor) -> Distribution:
        """return the distribution `p(x|z)`"""
        px_loc = self.decoder(z)
        px_loc = px_loc.view(-1, *self.input_shape) # reshape the output #old
        shape = torch.ones(px_loc.shape)
        shape = shape.to(self.device)
        shape = 0.05 * shape 
        #sandsynlighedsfordeling der giver 1 eller 0, baseret på log-odds givet i logits input fra p(x|z).
        #Dvs. at px_logits angiver sandsynligheden for at det givne pixel er henholdsvist rød,grøn,blå. Pixel værdien
        #er enten 0 eller 1. Når man sampler fra bernoulli fordelingen fås dermed et billede, som givet z, giver en figur,
        #som er bestemt af de sandsynligheder der er i px_logits (p(x|z)). Dvs. at for et givet latents space, kan en
        #figur/et tal reproduceres ud fra de beregnede sandsynligheder og den efterfølgende sample fra Bernoulli fordelingen.
        #return Bernoulli(logits=px_loc, validate_args=False)
        return Normal(loc=px_loc, scale = shape)
        

    def forward(self, x, y, e) -> Dict[str, Any]:
        """compute the posterior q(z|x) (encoder), sample z~q(z|x) and return the distribution p(x|z) (decoder)"""
        
        #### Run through ENCODER and calculate mu and sigma for latent space sampling
        # define the posterior q(z|x) / encode x into q(z|x)
        qz = self.posterior(x, y, e)
        
        # sample the posterior using the reparameterization trick: z ~ q(z | x)
        #### LATENT SPACE
        z = qz.rsample()
        
        #### DECODER
        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)

        return {'px': px, 'qz': qz, 'z': z}

    def reduce(self, x:Tensor) -> Tensor:
        """for each datapoint: sum over all dimensions"""
        return x.view(x.size(0), -1).sum(dim=1)

    def VariationalInference(self, x, y, e):
        
        # forward pass through the model to get the encoder and decoder outputs
        parameters_and_latent_space = self.forward(x, y, e)

        # unpack encoder parameters from (px), decoder parameters (qz) and the latent space (z)
        px_ze, qz_xye, z = [parameters_and_latent_space[k] for k in ["px", "qz", "z"]]


        # DEFINE THE PRIOR p(z)
        #### PRIOR
        z_temp = z.detach().requires_grad_(requires_grad = True)
        y_temp = y.detach()
        e_temp = e.detach()
        TNN_parameters, LambdaNN_parameters, lambdaf_parameters = self.parameters_for_non_factorized_prior(z_temp, y_temp, e_temp) #bruger temp da priors træning ikke skal påvirke encoder og decoder

        #prior calculation from page 28 (til differentiering ifm. eq. 64 skal dette ikke være logget, men når
        # elbo loss beregnes, skal man huske at tage log at denne værdi jvf. p. 28)

        #According to page 6. equation (8)-(10)
        TNN_parameters_hat = TNN_parameters.detach()
        LambdaNN_parameters_hat = LambdaNN_parameters.detach()
        lambdaf_parameters_hat = lambdaf_parameters.detach()

        log_pz_ye_ELBO =  self.non_factorized_prior_pz(z, TNN_parameters_hat, LambdaNN_parameters_hat, lambdaf_parameters_hat)
        log_pz_ye_SM = self.non_factorized_prior_pz(z_temp, TNN_parameters, LambdaNN_parameters, lambdaf_parameters) #phi_hat (or phi.detach()) from page. 6 is implemented implicitly/auto-
        #matically through the implementation of SM on page 28 eq. (64), where autograd of p_T,lambda is used instead of the conditional distribution q_phi(Z|X,Y,E), here phi
        #is evidently excluded from the gradient calculation.
        #OBS: HER SKAL DER IKKE ANVENDES LOG-PROBS AF PZ_YE JVF. SM DELEN AF EQ. (64) PÅ SIDE 28... eller hvad???
        dpdz_ye = torch.autograd.grad(log_pz_ye_SM.sum(), z_temp, create_graph = True, retain_graph=True)[0]

        #ddpz_ye = torch.autograd.grad(dpz_ye.mean(), z_temp, create_graph = True, retain_graph=True)[0] #changed sum() to mean()
        ddpdz_sq_ye = torch.autograd.grad(dpdz_ye.sum(), z_temp, create_graph = True, retain_graph=True)[0] #original

        #### SM loss SM loss SM loss SM loss SM loss SM loss SM loss SM loss ####
        #Calculation from page 28 equation 64
        SM = (ddpdz_sq_ye + 0.5 * dpdz_ye.pow(2)).sum(1)
        #### SM loss SM loss SM loss SM loss SM loss SM loss SM loss SM loss ####

        #### ELBO loss ELBO loss ELBO loss ELBO loss ELBO loss ELBO loss ELBO loss ####
        # evaluate log probabilities
        # Skal jvf. s. 32 ændres til at være en normal fordeling i stedet for en
        # bernoulli fordeling, og her skal mean være outputtet af decoderen og varians skal
        #blot sættes til at være 0.01
        
        log_px_ze = self.reduce(px_ze.log_prob(x)) #log(p(x|z)): Sandsynligheden for at observere vores input variabel x
        #givet vores latent space (tjekker modellens evne til at rekonstruere sig selv, ved at maximere sandsynlig-
        #heden for at observere inputtet selv, givet det konstruerede latent space.
        
        ####(old)log_pz = reduce(pz.log_prob(z)) #log(p(z)): 
        #log_pz_ye = torch.log(pz_ye)#Sandsynligheden for at observere vores latent space, givet at
        #latent space følger en standard-normal fordeling (Jo højere sandsynlighed jo bedre)
        
        log_qz_xye = self.reduce(qz_xye.log_prob(z)) #log(q(z|x)): Sandsynligheden for at generere netop dette latent space givet 
        #vores input billede x. Denne værdi skal helst være lav?
        
        # compute the ELBO: 
        # `L^\beta = E_q [ log p(x|z) ] - \beta * D_KL(q(z|x) | p(z))`
        # where `D_KL(q(z|x) | p(z)) = log q(z|x) - log p(z)`
        #########################################################################################################
        # Reconstruction loss: E_q [ log p(x|z) ]
        # Regularization term: \beta * D_KL(q(z|x) | p(z))` => Forsøger at tvinge fordelingen q(z|x) mod N(0,1)?
        #########################################################################################################
        
        kl = log_qz_xye - log_pz_ye_ELBO
        
        elbo = log_px_ze - kl
        ####
        
        # loss
        loss = -(elbo.mean() - SM.mean())
        #print("SM: {}. elbo: {} = log_px_ze: {} + log_pz_ye: {} - log_qz_xye: {}".format(SM.mean(), elbo.mean(),log_px_ze.mean(),pz_ye.mean(),log_qz_xye.mean()))

        # prepare the output
        with torch.no_grad():
            diagnostics = {'elbo': elbo, 'log_px':log_px_ze, 'kl': kl}
        
        outputs = parameters_and_latent_space
        return loss, diagnostics, outputs

    #def sample_from_prior(self, batch_size:int=100):
    #    """sample z~p(z) and return p(x|z)"""
    #    
    #   # Laver bare reconstruction baseret på latent space
    #    #Kan evt. fjernes. Anvendes bare til at vise hvor god modellen er til at generere data baseret på
    #    #latent space genererede data. Funktionen anvendes kun i make_vae_plots.
    #    
    #    # degine the prior p(z)
    #    pz = self.prior(batch_size=batch_size)
    #    
    #    # sample the prior 
    #    z = pz.rsample()
    #    
    #    # define the observation model p(x|z) = B(x | g(z))
    #    px = self.observation_model(z)
    #    
    #    return {'px': px, 'pz': pz, 'z': z}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f">> Using device: {device}")

latent_features = 10 #Husk at opdater denne parameter nede i 'initialization', hvis den skal bruges i VAE loopet også
batch_size = 256
NF_ivae = NF_iVAE(images[0].shape, latent_features, device)
NF_ivae = NF_ivae.to(device)
print(NF_iVAE)

In [ ]:
from collections import defaultdict
from tqdm import tqdm
# define the models, evaluator and optimizer

# VAE
latent_features = 10 #Hyper parameter

# The Adam optimizer works really well with VAEs.
optimizer = torch.optim.Adam(NF_ivae.parameters(), lr=1e-4) #Hyper parameter, tilføj evt. weight_decay (L2 regularization)

# define dictionary to store the training curves
training_data = defaultdict(list)
validation_data = defaultdict(list)

epoch = 0

In [ ]:
torch.manual_seed(1)
num_epochs = 100 #hyper parametre
#batch size hyper parameter can be changed in the dataloader in the beginning.

# move the model to the device 
sample_counter = 0
# training..
while epoch < num_epochs:
    epoch+= 1
    training_epoch_data = defaultdict(list)
    NF_ivae.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for sample in tqdm(train_loader): #tqdm
        sample_counter += 1
        
        x = sample['image']
        x = x.to(device)

        y = sample['label'].reshape(-1,1)
        y = y.to(device)

        e = sample['environment'].reshape(-1,1)
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        optimizer.zero_grad()
        loss, diagnostics, outputs = NF_ivae.VariationalInference(x, y, e)
        loss.backward()
        optimizer.step()
        
        # gather data for the current bach
        for k, v in diagnostics.items():
            training_epoch_data[k] += [v.mean().item()]
            

    # gather data for the full epoch
    for k, v in training_epoch_data.items():
        training_data[k] += [np.mean(training_epoch_data[k])]

    px = outputs['px']
    samples = px.sample()

    xplot = x[0].to(torch.device("cpu"))
    samplesplot = samples[0].to(torch.device("cpu"))

    #fig, axes = plt.subplots(2,2)
    #show_CMNIST(xplot, label=int(y[0]), environment=int(e[0]), ax = axes[0,0])
    #show_CMNIST(samplesplot, label=int(y[0]), environment=int(e[0]),  ax = axes[0,1])
    #axes[1,0].plot(training_data['elbo'], label = "elbo")
    #display(fig)
    #clear_output(wait = True)


    #if (epoch+1)%(num_epochs*0.1) == 0:
        #print("epoch: {}: training loss: elbo = {}, kl = {}, log_px = {}, total loss = {}".format(epoch, training_data['elbo'][epoch-1], training_data['kl'][epoch-1], training_data['log_px'][epoch-1]), loss)
    
    # Reproduce the figure from the begining of the notebook, plot the training curves and show latent samples
    #make_vae_plots(ivae, x, y, outputs, training_data, validation_data)

In [ ]:
#torch.save(NF_ivae.state_dict(), '100epochs_NF_iVAE_var005_flipped_v3.pt')

In [ ]:
NF_ivae.load_state_dict(torch.load('100epochs_NF_iVAE_var005_flipped_v3.pt', map_location=torch.device(device)))
NF_ivae = NF_ivae.to(device)

In [ ]:
#Dummy code for the testing of the PC algorithm to ensure the model data is prepared in the right format
import numpy as np
import pandas as pd
import cdt
cdt.SETTINGS.rpath = 'C:/Program Files/R/R-4.2.2/bin/Rscript' # this path should point to your own R implementation !
from cdt.causality.graph import PC
import networkx as nx
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

device = torch.device("cpu")
print(f">> Using device: {device}")

#cmnist = ConcatDataset([cmnist_data1, cmnist_data2])
#loader = DataLoader(cmnist, batch_size = 40000, batch_sampler = None)

sample = next(iter(loader))

x = sample['image']
y = sample['label'].reshape(-1,1)
e = sample['environment'].reshape(-1,1)

x = x.to(device)
y = y.to(device)
e = e.to(device)

NF_ivae = NF_ivae.to(device)

output = NF_ivae.forward(x, y, e)

z = output['z']

z_full = z
x_full = x
y_full = y
e_full = e

In [ ]:
qz = output['qz']
z_sample1 = qz.rsample()
z_sample2 = qz.rsample()
z_sample3 = qz.rsample()
z_sample4 = qz.rsample()
z_sample5 = qz.rsample()
z_sample6 = qz.rsample()
z_sample7 = qz.rsample()
z_sample8 = qz.rsample()
z_sample9 = qz.rsample()
z_sample10 = qz.rsample()

y = y.detach()
e = e.detach()



y_df = pd.DataFrame(torch.cat((y,y,y,y,y,y,y,y,y,y), dim = 0))
z_df = pd.DataFrame(torch.cat((z_sample1.detach(),z_sample2.detach(),z_sample3.detach(),z_sample4.detach(),z_sample5.detach(),
    z_sample6.detach(),z_sample7.detach(),z_sample8.detach(),z_sample9.detach(),z_sample10.detach(),), dim = 0))
df = y_df#; df = z_df; #df = z_df;  
df['Z1'] = z_df[0]
df['Z2'] = z_df[1]
df['Z3'] = z_df[2]
df['Z4'] = z_df[3]
df['Z5'] = z_df[4]
df['Z6'] = z_df[5]
df['Z7'] = z_df[6]
df['Z8'] = z_df[7]
df['Z9'] = z_df[8]
df['Z10'] = z_df[9]


df.columns = ['Y','Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8', 'Z9', 'Z10']
df.head()

df.shape

In [ ]:
glasso = cdt.independence.graph.Glasso()

skeleton = glasso.predict(df)

#model_pc = cdt.causality.graph.PC(alpha = 0.2)

from causallearn.search.ConstraintBased.PC  import pc

from causallearn.utils.cit import fisherz
cg = pc(np.array(df),  0.05, fisherz, True, 0, 0)

cg.draw_pydot_graph()
#graph_pc = model_pc.predict(df, skeleton)

In [ ]:
df

In [ ]:
def find_parents(pc_array,y_pos,z_pos):
    ind = np.where(pc_array[z_pos,y_pos] == 1)
    ind_DAGpar = []
    ind_par = []
    for i in ind[0]:
        if pc_array[y_pos,i] == 0:
            ind_DAGpar.append(i)
        else:
            ind_par.append(i)
    return ind_DAGpar, ind_par

pc = PC(alpha=0.05)
pc_output = pc.predict(df)

nx.draw(pc_output, with_labels=True, font_weight='bold', font_size = 16, node_size = 9000)
plt.savefig('PC_iVAE_alpha_005.png', transparent=True)
plt.show()



In [ ]:
pc_array = pc._run_pc(df)
parents = torch.tensor(find_parents(pc_array, 0, range(0,10))[0]).to(device)
parents = parents - 1
print(parents)

In [ ]:
#Found indices:
 
#parents = parents.to("cpu")
parents = torch.tensor([1,2])
options = np.array([0,1,2,3,4,5,6,7,8,9])
children = np.delete(options, parents)
nParents = len(parents)
nChildren = 10 - nParents

In [ ]:
print(children)

In [ ]:
#Accuracy function from week 4:
def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

#Building a classifier:

class ClassifierNF_iVAE(nn.Module):
    def __init__(self, indices):
        super().__init__()
        self.indices = indices
        #Layers
        self.input_layer = nn.Linear(in_features=2, out_features = 50)
        self.layer1 = nn.Linear(in_features = 50, out_features = 100)
        self.output_layer = nn.Linear(in_features = 100, out_features = 1)

        #activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid() 
    #forward function(self, x):
    def forward(self, x):
        
        x = self.input_layer(x)
        x = self.relu(self.layer1(x))
        x = self.sigmoid(self.output_layer(x))

        return x
    # 

classifierNF_iVAE = ClassifierNF_iVAE(parents)
classifierNF_iVAE = classifierNF_iVAE.to(device)

In [ ]:
NF_ivae = NF_ivae.to(device)
#indices = indices.to(device)
parents = parents.to(device)
classifierNF_iVAE = classifierNF_iVAE.to(device)

#Set iVAE to eval - the no.grad thing is not an issue as we are only using the encoder.
NF_ivae.eval()

#TRAINING THE CLASSIFIER
epoch = 0
num_epochs = 10 #hyper parametre
#batch size hyper parameter can be changed in the dataloader in the beginning.

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f">> Using device: {device}")

# training..
training_epoch_data = []
test_epoch_data = []
test_acc, train_acc = [], []

while epoch < num_epochs:
    epoch+= 1
    training_batch_data = []
    test_batch_data = []
    classifierNF_iVAE.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label'].to(torch.float32)
        y = y.to(device)

        e = sample['environment'].to(torch.float32)
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = NF_ivae.posterior(x, y.reshape(-1,1), e.reshape(-1,1))
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifierNF_iVAE(z).view(-1)

        loss = loss_fn(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # gather data for the current batch
        training_batch_data.append(loss)
            

    # gather data for the full epoch
    training_epoch_data.append(torch.tensor(training_batch_data).mean())
    print("Epoch: {}, Mean epoch loss: {}".format(epoch, torch.tensor(training_batch_data).mean()))

    

In [ ]:
#NF_ivae = NF_ivae.to(device)
#indices = indices.to(device)
#parents = parents.to(device)

#Set iVAE to eval - the no.grad thing is not an issue as we are only using the encoder.
#NF_ivae.eval()

#TRAINING THE CLASSIFIER
epoch = 0
#batch size hyper parameter can be changed in the dataloader in the beginning.

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f">> Using device: {device}")

# training..
training_epoch_data = []
test_epoch_data = []
test_acc, train_acc = [], []


#Evaluate training
with torch.no_grad():
    classifierNF_iVAE.eval()
    train_targs, train_preds = [], []
    for sample in train_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label']
        y = y.to(device)

        e = sample['environment']
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = NF_ivae.posterior(x, y.reshape(-1,1), e.reshape(-1,1))
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifierNF_iVAE(z).view(-1)
        prediction = torch.round(output)

        train_targs += list(y.cpu().numpy())
        train_preds += list(prediction.data.cpu().numpy())

    #Evaluating validation
    val_targs, val_preds = [], []
    for sample in test_loader:
        x = sample['image']
        x = x.to(device)

        y = sample['label']
        y = y.to(device)

        e = sample['environment']
        e = e.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        qz = NF_ivae.posterior(x, y.reshape(-1,1), e.reshape(-1,1))
        z = qz.rsample()
        z = torch.index_select(z, 1, parents)

        #Get classification prediction
        output = classifierNF_iVAE(z).view(-1)
        prediction = torch.round(output)


        val_targs += list(y.cpu().numpy())
        val_preds += list(prediction.data.cpu().numpy())

train_acc_cur = accuracy(Tensor(train_targs),Tensor(train_preds))
train_acc.append(train_acc_cur)

test_acc_cur = accuracy(Tensor(val_targs),Tensor(val_preds))
test_acc.append(test_acc_cur)
print("Epoch %2i : , Train acc %f, Valid acc %f" % (
            epoch, train_acc_cur, test_acc_cur))



In [ ]:
pd.DataFrame([np.array(y),np.array(prediction)]).T

In [ ]:
np.where(prediction == 0)

In [ ]:
np.where(prediction == 1)

In [ ]:
np.where(y == 1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")

test_data_full = DataLoader(cmnist_test_data, batch_size = 20000, batch_sampler = None)

sample = next(iter(test_data_full))

x_test = sample['image']
y_test = sample['label'].reshape(-1,1)
e_test = sample['environment'].reshape(-1,1)

x_test = x_test.to(device)
y_test = y_test.to(device)
e_test = e_test.to(device)

NF_ivae = NF_ivae.to(device)
output = NF_ivae.forward(x_test, y_test, e_test)

classifierNF_iVAE = classifierNF_iVAE.to(device)

z_test = output['z']

z_full = z_test.detach()
x_full = x_test.detach()
y_full = y_test.detach()
e_full = e_test.detach()

Zp = torch.randn(z_full.shape[0], nParents)
Zc = torch.randn(z_full.shape[0], nChildren)
Z = torch.randn(z_full.shape)



Zp = Zp.to(device)
Zc = Zc.to(device)
Z = Z.to(device)

loss_BCE = nn.BCELoss()

Zp.requires_grad = True
Zc.requires_grad = True

optimizer = torch.optim.Adam(params=[Zp, Zc], lr=1e-1)

def get_reconstruction(Zp, Zc):
    Z[:,parents] = Zp
    Z[:,children] = Zc
    return(NF_ivae.observation_model(Z))

loss_values = []


for i in tqdm(range(num_epochs)):
    sample = next(iter(test_data_full))
    x = sample['image']
    x = x.to(device)

    px = get_reconstruction(Zp,Zc)
    loss = -px.log_prob(x)
    loss = loss.mean()
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

    loss_values.append(loss.detach().cpu())
    #print(Z[0,:])
    #print("Step done")
    #print(loss)

plt.plot(loss_values)
plt.show()